In [1]:
#@author - Tejasvi Sharma
#NLP HW1
import nltk.data
from os import listdir
from os.path import isfile, join
from nltk.util import bigrams 
from nltk.tokenize import TreebankWordTokenizer
import re

In [2]:
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
treebank_tokenizer = TreebankWordTokenizer()

#
dir_base = "./data"


def read_file(filename):
    input_file_text = open(filename , encoding='utf-8').read()
    return input_file_text

#this function will read all the files in the directory and store data in a list one by one by file.   
def read_directory_files(directory):
    file_texts = []
    files = [f for f in listdir(directory) if isfile(join(directory, f))]
    for f in files:
        file_texts.append(read_file(join(directory, f) ))
    return file_texts
    
#text_corpus has all the document data in lists
text_corpus = read_directory_files(dir_base)

#this function will read the text data and tokenize it first by sentence then by word in that sentence.
def read_text_and_tokenize(text_data):
    punkt_sentences = sentence_tokenizer.tokenize(text_data)
    sentences_words = [treebank_tokenizer.tokenize(sentence) for sentence in punkt_sentences]
    return sentences_words

#this function will parse the sentence and get value of an acronym.
def get_value(acronym,sentence):
    acr=acronym.replace(".","").replace("&","A")
    if("-" in acr):
        o=acr.index("-")
        acr=acr[0:o]+acr[o+2:] #for acronyms like ACC-RI

        
    for i in range(len(sentence)):
        if(str.lower(sentence[i][0])==str.lower(acr[0])):
            value=""
            value=value+sentence[i]+" "
            n=len(acr)
            ind=1 # as first is matched
            j=i+1 #counter for words
            flag=1
            while(ind<n):
                if(str.lower(sentence[j][0])==str.lower(acr[ind])):
                    value=value+sentence[j]+" "
                    j=j+1
                    ind=ind+1
                else:
                    flag=0
                    break;
            if(flag==1):
                return(value.strip())

In [3]:
#For now I have assumed, full form of an acronym is only in that sentence

#dictionary for saving acronym document by document
di_doc={} #doc_number -> ("Short Form","Long Form"), this is for output
#dictionary to save all the acronyms i.e for corpus.
di_corp={} #acronym -> value
for i in range(len(text_corpus)):
#for i in range(0,13):
    text=text_corpus[i]
    text_data=read_text_and_tokenize(text)
    pattern = re.compile("[A-Z.&-]{2,}|^[A-Z][A-Za-z]*[A-Z]$")
    for sentence in text_data:
        for words in sentence:
            found=pattern.fullmatch(words)
            if(found!=None):
                #print(found.group(0))
                if(found.group(0) not in list(di_corp.keys())):
                    di_corp[found.group(0)]=get_value(found.group(0),sentence)
            
                #getting the value from whole corpus for a acronym
                if(i not in list(di_doc.keys())):
                    di={}
                    di[found.group(0)]=di_corp[found.group(0)]
                    di_doc[i]=di
                else:
                    word_dict=di_doc[i]
                    word_dict[found.group(0)]=di_corp[found.group(0)]
                    di_doc[i]=word_dict
                    
            else:
                pass #try to find in whole document of continous words.
                    #

In [4]:
#To convert to a csv file
import pandas as pd
csv_dict={"Document_num":[],"acr":[],"value":[]}
for k,v in di_doc.items():
    for a,j in v.items():
        csv_dict["Document_num"].append(k)
        csv_dict["acr"].append(a)
        csv_dict["value"].append(j)
    

In [5]:
pd.DataFrame(csv_dict)

,Document_num,acr,value
0,0,FY,None
1,0,P.C.,None
2,1,DoD,None
3,2,DoD,None
4,3,U.S.,None
5,4,DoD,None
6,5,ISMA,None
7,5,IG,None
8,5,DoD,None
9,5,DCIE,None
